In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging

In [3]:
tf.autograph.set_verbosity(0)
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [15]:
## Synthetic Data
rng = np.random.default_rng(2)
xtr = rng.random(400).reshape(-1,2)
xtr[:,1] = xtr[:,1] * 4 + 11.5         
xtr[:,0] = xtr[:,0] * (285-150) + 150  
ytr = np.zeros(len(xtr))

i=0
for t,d in xtr:
    y = -3/(260-175)*t + 21
    if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
        ytr[i] = 1
    else:
        ytr[i] = 0
    i += 1

ytr = ytr.reshape(-1,1)

In [16]:
print(xtr.shape)
print(ytr.shape)

(200, 2)
(200, 1)


In [17]:
print(f"Temp Max: {np.max(xtr[:,0]):0.2f}, Min: {np.min(xtr[:,0]):0.2f}")
print(f"Duration Max: {np.max(xtr[:,1]):0.2f}, Min: {np.min(xtr[:,1]):0.2f}")

Temp Max: 284.99, Min: 151.32
Duration Max: 15.45, Min: 11.51


In [18]:
normalizing_layer = tf.keras.layers.Normalization()
normalizing_layer.adapt(xtr)
norm_xtr = normalizing_layer(xtr)

print(f"Temp Max: {np.max(norm_xtr[:,0]):0.2f}, Min: {np.min(norm_xtr[:,0]):0.2f}")
print(f"Duration Max: {np.max(norm_xtr[:,1]):0.2f}, Min: {np.min(norm_xtr[:,1]):0.2f}")

Temp Max: 1.66, Min: -1.69
Duration Max: 1.79, Min: -1.70


In [19]:
Xt = np.tile(norm_xtr,(1000,1))
Yt= np.tile(ytr,(1000,1))   
print(Xt.shape, Yt.shape)  

(200000, 2) (200000, 1)


In [21]:
tf.random.set_seed(1234)
model = Sequential([
    tf.keras.layers.Input(shape=(2,)),
    Dense(3, activation="sigmoid", name="L1"),
    Dense(1, activation="sigmoid", name="L2")
])

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ L1 (Dense)                      │ (None, 3)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
W1, b1 = model.get_layer('L1').get_weights()
W2, b2 = model.get_layer('L2').get_weights()
print(f"Shape: L1 W {W1.shape}, L1 B {b1.shape}, L2 W {W2.shape}, L2 B {b2.shape}")
print(f"Values: L1 W {W1}, L1 B {b1}, L2 W {W2}, L2 B {b2}")

Shape: L1 W (2, 3), L1 B (3,), L2 W (3, 1), L2 B (1,)
Values: L1 W [[-0.93967783  0.6296128   1.0516868 ]
 [-0.9623561   0.03163135 -0.11909634]], L1 B [0. 0. 0.], L2 W [[-0.3527357 ]
 [-0.34983557]
 [-0.8882197 ]], L2 B [0.]


In [25]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
)

model.fit(Xt, Yt, epochs=10)

Epoch 1/10


I0000 00:00:1751689351.981995    1942 service.cc:152] XLA service 0x72d8e0016230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751689351.982047    1942 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-07-05 09:22:32.039000: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751689352.198950    1942 cuda_dnn.cc:529] Loaded cuDNN version 90701


  44/6250 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.5237   

I0000 00:00:1751689352.907660    1942 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6250/6250 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.2418
Epoch 2/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.1198
Epoch 3/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.0550
Epoch 4/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.0175
Epoch 5/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 0.0112
Epoch 6/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.0077
Epoch 7/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.0055 
Epoch 8/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.0039
Epoch 9/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.0028
Epoch 10/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.0021


In [26]:
W1, b1 = model.get_layer("L1").get_weights()
W2, b2 = model.get_layer("L2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[-11.153579    14.263588    -0.12538654]
 [ -0.20745812  11.915943    -8.964384  ]] 
b1: [-12.188365    1.7975255 -11.182624 ]
W2:
 [[-50.580147]
 [-43.45739 ]
 [-46.35238 ]] 
b2: [26.501942]


In [27]:
X_test = np.array([
    [200,13.9],  # positive example
    [200,17]])   # negative example
X_testn = normalizing_layer(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
predictions = 
 [[9.7701812e-01]
 [4.2240067e-08]]


In [29]:
yhat = (predictions >= 0.5).astype(int)
yhat

array([[1],
       [0]])